## Building a stadium
A town council wishes to construct a small stadium in order to improve the services provided to the people living in the district. After the invitation to tender, a local construction company is awarded the contract and wishes to complete the task within the shortest possible time. All the major tasks are listed in the following table. Some tasks can only start after the completion of certain other tasks, as indicated by the "Predecessors" column.

![](stad1.png)

**a)** What is the earliest possible date of completion for the construction? Note that the last two columns of the table are not relevant for this part of the problem.

**b)** The town council wants the builder to expedite the project. As an incentive, the council will pay a bonus of $\$30$k/week for each week the work finishes early. To accomplish this, the builder may employ additional workers and rent more equipment to cut down on the total time. The last two columns of the table show the maximum number of weeks that can be saved per task and the associated additional cost per week incurred by the extra work. When will the project be completed if the builder is acting in a way that maximizes his profit?

In [29]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        using JuMP, GLPK, LinearAlgebra
tasks = 1:18
durations = [2 16 9 8 10 6 2 2 9 5 3 2 1 7 4 3 9 1]
predecessors = ([], [1], [2], [2], [3], [4,5], [4], [6], [4,6],
                [4], [6], [9], [7], [2], [4,14], [8,11,14], [12], [17])
comp = zeros(1,18)       # week complete of each task
function minweek(dur)
    for i=1:18
        plus = 0
        for j in predecessors[i]
            if plus<comp[j]
                plus = comp[j]
            end
        end
        comp[i] = dur[i] + plus
    end
    return(maximum(comp))
end
println(typeof((minweek(durations))))
# print(maximum(comp))

######## using linear programming
# m = Model(with_optimizer(GLPK.Optimizer))
# @variable(m, comp[1:18]>=0)
# for i=1:18
#     if size(predecessors[i])[1] == 0
#         @constraint(m, comp[i] >= durations[i])
#     elseif size(pre[i])[1] == 1
#         @constraint(m, comp[i] >= durations[i] + comp[predecessors[i]])
#     else
#         for j in predecessors[i]
#             @constraint(m, comp[i] >= durations[i] + comp[j])
#         end
#     end
# end
# @objective(m, Min, sum(comp[i]))
# optimize!(m)
# result = zeros(1,18)
# for i=1:18
#     println(value(comp[i]))
#     result[i] += value(comp[i])
# end
# print(maximum(result))

# additional columns of data (maximum reduction possible )
max_reduction =  [0,  3,  1,  2,  2,  1, 1, 0,  2,  1,  1, 0, 0,  2,  2, 1,  3, 0] 
cost_reduction = [0, 30, 26, 12, 17, 15, 8, 0, 42, 21, 18, 0, 0, 22, 12, 6, 16, 0]   
println(size(max_reduction))
println(size(cost_reduction))
bonus_amount = 30     # bonus for expediting the project ($1,000/week )
m = Model(with_optimizer(GLPK.Optimizer))
@variable(m, reduction[1:18]>=0)      # actual reduction weeks for each task
@variable(m, 0<=t<=63)
@variable(m, totalSum >= 0)
@variable(m, dur[1:18])
for i=1:18
    @constraint(m, reduction[i] <= max_reduction[i])
end
println(size(reduction))
@constraint(m, totalSum == ((cost_reduction')*reduction))
@objective(m, Max, 30*(63-t) - totalSum)
# println(typeof(minweek(durations-reduction')))
@constraint(m, dur .== durations - reduction')
@constraint(m, t==(minweek(dur)))
@constraint(m, 30*(63-t) >= sum(cost_reduction.*reduction))
# m

Float64
(18,)
(18,)
(18,)


MethodError: MethodError: Cannot `convert` an object of type GenericAffExpr{Float64,VariableRef} to an object of type Float64
Closest candidates are:
  convert(::Type{T<:Number}, !Matched::T<:Number) where T<:Number at number.jl:6
  convert(::Type{T<:Number}, !Matched::Number) where T<:Number at number.jl:7
  convert(::Type{T<:Number}, !Matched::Base.TwicePrecision) where T<:Number at twiceprecision.jl:250
  ...

## Museum site planning

A site is being investigated as a potential location for a new museum. An aerial plan of the site is shown in the figure below (in units of feet). The museum will have a circular footprint and law mandates that there be at least 50 feet of clearance between the building and any edge of the site. If we want the largest possible museum, where should it be located? What is its optimal radius?

![](museum.png)

In [7]:
## your code goes here
# -x <= 0 # left
# y <= 500 # up
# -y <= 0 # down
# 3x - y <= 1500 bottom
# 2/3x + y <=  700 top right
# max(min(r)); a'y + norm(a)r = b
using GLPK, LinearAlgebra, JuMP
m = Model(with_optimizer(GLPK.Optimizer))
a = [-1 0; 0 1; 0 -1; 3 -1; 2/3 1]
b = [0; 500; 0; 1500; 700]
@variable(m, y[1:2])
@variable(m, r>=0)
for i=1:5
    @constraint(m, a[i,:]'*y + norm(a[i,:])*r <= b[i])
end
@objective(m, Max, r)
m
optimize!(m)
println(value(r))

244.02852679380186


## Virginia Prestress (VP) location
VP was planning for production of a new product: concrete utility poles. That production required a new concrete casting area to make poles and a storage area for finished products.

Figure bellow presents the implied facilities location problem for our fictitious case. The two new facilities will interact with each other and with three existing operations: the concrete batching facility, where premixed concrete is prepared, the steel area, where reinforcing steel is manufactured, and the shipping gate, where finished poles are processed out of the plant. A coordinate system quantifies the locations of all three existing facilities, and the adjoining table displays material handling costs of expected traffic between facilities. For example, each foot of distance between the pole storage area and the shipping gate adds $\$0.40$ in crane activity. We must choose locations for the new facilities to minimize total material handling cost. Note that the facilities design use [Manhattan distance](https://vi.wikipedia.org/wiki/Kho%E1%BA%A3ng_c%C3%A1ch_Manhattan).

![](vp1.png)

In [1]:
## your code goes here
using GLPK, JuMP, LinearAlgebra
m = Model(with_optimizer(GLPK.Optimizer))
concrete = [300, 1200]
steel = [0, 600]
shipping = [600, 0]
cost = [4,4,1.1,1.1,0.7,0.7,0.65,0.65,0.4,0.4]
@variable(m, x[1:2] >= 0) #casting coordinate
@variable(m, y[1:2] >= 0) #storage coordinate
@variable(m, t[1:10]) #t
@objective(m, Min, sum(cost.*t))

@constraint(m, x[1] - y[1] >= 1)
@constraint(m, -x[1] + y[1] <= -1)

@constraint(m, x[2] - y[2] >= 1)
@constraint(m, -x[2] + y[2] <= -1)

@constraint(m, t[1] >= x[1] - y[1])
@constraint(m, t[1] >= -x[1] + y[1])

@constraint(m, t[2] >= x[2] - y[2])
@constraint(m, t[2] >= -x[2] + y[2])

@constraint(m, t[3] >= x[1] - concrete[1])
@constraint(m, t[3] >= -x[1] + concrete[1])

@constraint(m, t[4] >= x[2] - concrete[2])
@constraint(m, t[4] >= -x[2] + concrete[2])

@constraint(m, t[5] >= x[1] - steel[1])
@constraint(m, t[5] >= -x[1] + steel[1])

@constraint(m, t[6] >= x[2] - steel[2])
@constraint(m, t[6] >= -x[2] + steel[2])

@constraint(m, t[7] >= y[1] - steel[1])
@constraint(m, t[7] >= -y[1] + steel[1])

@constraint(m, t[8] >= y[2] - steel[2])
@constraint(m, t[8] >= -y[2] + steel[2])

@constraint(m, t[9] >= y[1] - shipping[1])
@constraint(m, t[9] >= -y[1] + shipping[1])

@constraint(m, t[10] >= y[2] - shipping[2])
@constraint(m, t[10] >= -y[2] + shipping[2])
m


A JuMP Model
Minimization problem with:
Variables: 14
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 4 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 22 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK
Names registered in the model: t, x, y

In [2]:
optimize!(m)
println(value(x[1]))
println(value(x[2]))
println(value(y[1]))
println(value(y[2]))
println(objective_value(m))
for i=1:10
    println(value(t[i]))
end

300.0
601.0
299.0
600.0
1432.3500000000001
1.0
1.0
0.0
599.0
300.0
1.0
299.0
0.0
301.0
600.0
